In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import os
import time
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [20]:
total = pd.read_csv('./data/가게합본.csv')
total.head()

,식당명,A_1_가게_분위기,A_2_주차,A_3_좌석,B_1_국물_스타일,B_2_잡내_유무,B_3_밥_제공_방식,B_4_주요_고기_부위_특징,B_5_순대_종류,B_6_내장_유무,C_1_김치_특징,C_2_깍두기_특징,C_3_특색_반찬,D_1_대표_추천_메뉴,D_2_리필_정책,D_3_가격대,주소,유명
0,60년 전통 할매국밥,노포/전통,주차지원,['입식 위주'],맑은/깔끔형,잡내 없음,토렴식,"['부드러운 식감', '푸짐한 고기 양']",정보 없음,내장 취급,정보 없음,정보 없음,['부추무침 제공'],기본 돼지국밥,"['국물 리필', '반찬 셀프바']",가성비(~9000원),동구 중앙대로533번길 4,1
1,경주박가국밥,동네 맛집,주차어려움,['입식 위주'],뽀얀/진한형,"['잡내 없음', '잡내 약간 있음']",따로국밥,"['푸짐한 고기 양', '부드러운 식감']",찰순대,내장 취급,익은 김치 스타일,"['달달한 맛', '새콤한 맛']",['소면 제공'],기본 돼지국밥,"['밥 리필', '반찬 셀프바']",가성비(~9000원),부산광역시 동래구 사직북로13번길 12,1
2,경주전통돼지국밥,노포/전통,주차지원,"['입식 위주', '좌식 있음']",뽀얀/진한형,잡내 없음,따로국밥,"['푸짐한 고기 양', '부드러운 식감']",찰순대,내장 취급,익은 김치 스타일,새콤한 맛,['소면 제공'],기본 돼지국밥,[],가성비(~9000원),부산시 북구 만덕2로44번길43,1
3,교통부돼지국밥,동네 맛집,주차어려움,"['입식 위주', '좌식 있음']",맑은/깔끔형,잡내 없음,"['따로국밥', '토렴식']","['삼겹/오겹', '부드러운 식감', '푸짐한 고기 양']","['찰순대', '고기/피순대']",내장 취급,"['겉절이 스타일', '익은 김치 스타일']","['달달한 맛', '새콤한 맛']",['양파절임 제공'],기본 돼지국밥,"['국물 리필', '밥 리필']",가성비(~9000원),부산광역시 부산진구 연지로 12-1 1층,1
4,남해돼지국밥,동네 맛집,주차지원,"['입식 위주', '좌식 있음']",뽀얀/진한형,잡내 없음,따로국밥,"['부드러운 식감', '푸짐한 고기 양']",찰순대,내장 취급,겉절이 스타일,달달한 맛,['소면 제공'],기본 돼지국밥,"['국물 리필', '밥 리필']",가성비(~9000원),부산광역시 해운대구 선수촌로 184,1


In [22]:
def collect_photo(search_word, place_name):
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')


    folder_name = './data/사진/'+ place_name
    # 폴더 생성
    os.makedirs(folder_name, exist_ok=True)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    search_url = f"https://map.naver.com/p/search/{search_word}"
    driver.get(search_url)

    time.sleep(4)
    driver.switch_to.frame('entryIframe')
    driver.find_element(By.XPATH, '//span[contains(text(), "사진")]').click()
    time.sleep(1)


    # 6. 스크롤 내리기 (더 많은 이미지 로딩)
    urls = []
    for _ in range(10):
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        photo_list = soup.find_all('div', class_ = 'wzrbN')
        for content in photo_list:
            url = content.find('img')['src']
            if url not in urls:
                urls.append(url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)


    print('사진 개수 : ' + str(len(urls))+'개')

    for i, url in enumerate(urls):
        try:
            urllib.request.urlretrieve(url, os.path.join(folder_name, f"{place_name}_{i}.jpg"))
        except:
            print(i, '사진 수집 실패', url)
            continue

    print('사진 수집 완료')
    driver.quit()

In [23]:
# '사진수집_여부'라는 새로운 컬럼을 만들고, 처음에는 모두 False로 설정
total['사진수집_여부'] = np.NaN

# 데이터프레임의 각 행(식당)에 대해 반복
for i in range(len(total)):
    # 현재 식당의 이름과 주소를 가져옴
    place_name = total.loc[i, '식당명']
    address = total.loc[i, '주소']
    
    # 사진을 검색할 때 사용할 후보 검색어들을 리스트로 정리
    search_keywords = [
        f"{place_name} {address}",  # 예: '가야돼지국밥 부산시 금정구'
        place_name,                 # 예: '가야돼지국밥'
        address                     # 예: '부산시 금정구'
    ]

    print(f"{i}번째: {place_name} 수집 시작")

    # 수집 성공 여부를 저장하는 변수
    collected = False

    # 후보 검색어들을 하나씩 시도
    for keyword in search_keywords:
        try:
            # 사진 수집 함수 호출
            collect_photo(keyword, place_name)
            
            # 성공하면 표시하고, 반복문 종료
            total.loc[i, '사진수집_여부'] = True
            print(f"  → 수집 성공: {keyword}")
            collected = True
            break  # 더 이상 시도하지 않고 빠져나감
        except Exception as e:
            print(f"  → 실패: {keyword} (오류: {e})")
            continue  # 다음 검색어로 넘어감

    # 세 가지 모두 실패한 경우
    if not collected:
        total.loc[i, '사진수집_여부'] = False
        print("최종 실패: 모든 키워드로도 수집 실패")

0번째: 60년 전통 할매국밥 수집 시작
사진 개수 : 100개
사진 수집 완료


C:\Users\user\AppData\Local\Temp\ipykernel_30100\1270396111.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  total.loc[i, '사진수집_여부'] = True


  → 수집 성공: 60년 전통 할매국밥 동구 중앙대로533번길 4
1번째: 경주박가국밥 수집 시작
사진 개수 : 100개
사진 수집 완료
  → 수집 성공: 경주박가국밥 부산광역시 동래구 사직북로13번길 12
2번째: 경주전통돼지국밥 수집 시작
  → 실패: 경주전통돼지국밥 부산시 북구 만덕2로44번길43 (오류: Message: entryIframe
)
사진 개수 : 88개
사진 수집 완료
  → 수집 성공: 경주전통돼지국밥
3번째: 교통부돼지국밥 수집 시작
  → 실패: 교통부돼지국밥 부산광역시 부산진구 연지로 12-1 1층 (오류: Message: entryIframe
)
  → 실패: 교통부돼지국밥 (오류: Message: entryIframe
)
  → 실패: 부산광역시 부산진구 연지로 12-1 1층 (오류: Message: entryIframe
)
최종 실패: 모든 키워드로도 수집 실패
4번째: 남해돼지국밥 수집 시작
사진 개수 : 100개
사진 수집 완료
  → 수집 성공: 남해돼지국밥 부산광역시 해운대구 선수촌로 184
5번째: 돌고래돼지국밥 수집 시작
  → 실패: 돌고래돼지국밥 부산시 남구 용소로 21번길 3 (오류: Message: entryIframe
)
사진 개수 : 100개
사진 수집 완료
  → 수집 성공: 돌고래돼지국밥
6번째: 두번째 늘해랑 수집 시작
사진 개수 : 100개
사진 수집 완료
  → 수집 성공: 두번째 늘해랑 부산진구 양지로 5번길 24
7번째: 마루돼지국밥 수집 시작
사진 개수 : 100개
사진 수집 완료
  → 수집 성공: 마루돼지국밥 부산광역시 금정구 개좌로 59-12
8번째: 명지돼지국밥 수집 시작
  → 실패: 명지돼지국밥 부산시 강서구 명지오션시티8로6번길 13 (오류: Message: entryIframe
)
  → 실패: 명지돼지국밥 (오류: Message: entryIframe
)
  → 실패: 부산시 강서구 명지오션시티8로6번길 13 (오류: Message: entryIframe
)
최종 실패: 

In [28]:
total[total['주소'].isna()]

,식당명,A_1_가게_분위기,A_2_주차,A_3_좌석,B_1_국물_스타일,B_2_잡내_유무,B_3_밥_제공_방식,B_4_주요_고기_부위_특징,B_5_순대_종류,B_6_내장_유무,C_1_김치_특징,C_2_깍두기_특징,C_3_특색_반찬,D_1_대표_추천_메뉴,D_2_리필_정책,D_3_가격대,주소,유명,사진수집_여부
25,쌍둥이돼지국밥,동네 맛집,주차어려움,"['입식 위주', '좌식 있음']",뽀얀/진한형,잡내 없음,따로국밥,"['삼겹/오겹', '항정살', '부드러운 식감', '푸짐한 고기 양']",찰순대,내장 취급,익은 김치 스타일,정보 없음,['양파절임 제공'],수육백반,['반찬 셀프바'],평균(10000원),NaN,1,False


In [34]:
total['사진수집_여부'].value_counts()

사진수집_여부
True     81
False    20
Name: count, dtype: int64

In [89]:
total[total['사진수집_여부'] == False][['식당명', '주소']]

,식당명,주소
11,바보국밥1호점,부산광역시 기장군 기장읍 차성로344번길 32 1층
15,부원돼지국밥,부산광역시 동래구 여고로91번길 90


In [108]:
idx = 46
place_name = total.loc[idx, '식당명']
address = total.loc[idx, '주소']
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


folder_name = './data/사진/'+ place_name
# 폴더 생성
os.makedirs(folder_name, exist_ok=True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

search_url = f"https://map.naver.com/p/search/{place_name}"
driver.get(search_url)

In [109]:
driver.switch_to.frame('entryIframe')
driver.find_element(By.XPATH, '//span[contains(text(), "사진")]').click()
time.sleep(1)

print(place_name)

# 6. 스크롤 내리기 (더 많은 이미지 로딩)
urls = []
for _ in range(10):
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    photo_list = soup.find_all('div', class_ = 'wzrbN')
    for content in photo_list:
        url = content.find('img')['src']
        if url not in urls:
            urls.append(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)


print('사진 개수 : ' + str(len(urls))+'개')

for i, url in enumerate(urls):
    try:
        urllib.request.urlretrieve(url, os.path.join(folder_name, f"{place_name}_{i}.jpg"))
    except:
        print(i, '사진 수집 실패', url)
        continue

print('사진 수집 완료')
total.loc[idx, '사진수집_여부'] = True
driver.quit()

가야공원돼지국밥
사진 개수 : 90개
사진 수집 완료


In [ ]:
# total.drop(index=[11, 15], inplace=True)
# total.reset_index(inplace=True, drop=True)
# total.to_csv('./data/가게합본_사진수집현황.csv')

In [119]:
restaurant_list = os.listdir('./data/사진')
for restaurant in restaurant_list:
        print(len(os.listdir('./data/사진/'+restaurant)))


100
90
99
100
88
33
100
95
100
62
66
67
100
100
94
100
91
96
100
28
89
86
100
90
95
95
47
90
95
88
98
95
95
100
15
100
90
100
100
99
100
100
96
48
90
100
82
100
94
100
96
90
73
100
98
91
99
86
90
100
100
75
86
90
100
86
100
100
100
99
100
98
87
12
90
87
98
89
86
56
90
100
100
100
84
86
87
100
79
100
100
70
100
100
58
100


In [120]:
total.to_csv('./data/가게합본_사진수집현황.csv', index=False)

In [1]:
import pandas as pd
total = pd.read_csv('./data/가게합본_사진수집현황.csv')

In [3]:
total['사진수집_여부']

0     True
1     True
2     True
3     True
4     True
      ... 
94    True
95    True
96    True
97    True
98    True
Name: 사진수집_여부, Length: 99, dtype: bool

In [41]:
import os
restaurant_list = os.listdir('./data/사진')

In [42]:
len(restaurant_list)

99

In [54]:
restaurant_list[75:][0], restaurant_list[75:][-1]

('진주돼지국밥', '흥부돼지국밥')

In [52]:
len(restaurant_list[25:50])

25

In [16]:
restaurant_list[0] == list(total['식당명'])[0]

True

In [26]:
total[total['식당명'].duplicated(keep = False)][['식당명', '주소']]

,식당명,주소
20,시골돼지국밥,부산시 남구 석포로20번길4
25,영진돼지국밥,부산광역시 강서구 명지오션시티8로 31
43,합천돼지국밥,부산광역시 금정구 서금로 17
69,시골돼지국밥,부산광역시 부산진구 골드테마길 54
72,영진돼지국밥,"부산광역시 부산진구 가야공원로 67, 1층"
93,합천돼지국밥,부산광역시 부산진구 엄광로44번길 36-2


In [39]:
place_name = '합천돼지국밥_일반'

options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


folder_name = './data/사진/'+ place_name
# 폴더 생성
os.makedirs(folder_name, exist_ok=True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

search_url = f"https://map.naver.com/p/search/{place_name[:-2]}"
driver.get(search_url)

In [40]:
driver.switch_to.frame('entryIframe')
driver.find_element(By.XPATH, '//span[contains(text(), "사진")]').click()
time.sleep(1)

print(place_name)

# 6. 스크롤 내리기 (더 많은 이미지 로딩)
urls = []
for _ in range(10):
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    photo_list = soup.find_all('div', class_ = 'wzrbN')
    for content in photo_list:
        url = content.find('img')['src']
        if url not in urls:
            urls.append(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)


print('사진 개수 : ' + str(len(urls))+'개')

for i, url in enumerate(urls):
    try:
        urllib.request.urlretrieve(url, os.path.join(folder_name, f"{place_name}_{i}.jpg"))
    except:
        print(i, '사진 수집 실패', url)
        continue

print('사진 수집 완료')
driver.quit()

합천돼지국밥_일반
사진 개수 : 57개
사진 수집 완료
